# 3 - Exemplo de Construção de Redes

In [3]:
import graph_tool_extras as gte
import pandas as pd
import csv

## Introdução

Para conduzir este notebook, vamos considerar o objetivo de construir uma rede baseada em partidas do *Campeonato Brasileiro de Futebol.*

Em particular, vamos considerar o arquivo abaixo, que pertence ao dataset [Campeonato Brasileiro de futebol](https://www.kaggle.com/datasets/adaoduque/campeonato-brasileiro-de-futebol), disponível no [Kaggle](https://www.kaggle.com/).

In [4]:
PATH = 'df_final.csv'

## Leitura dos dados em memória

Na melhor das situações, o arquivo é pequeno o suficiente para caber em memória e está em um formato suportado pelo [pandas](https://pandas.pydata.org/), como é justamente o caso do CSV.

In [5]:
df = pd.read_csv(PATH)

Nesse caso, podemos usar o método `head` para examinar as primeiras linhas e entender a organização dos dados.

In [6]:
df.drop(columns=["Unnamed: 0"], inplace=True)
df.head()

,user_id,movie_id,rating,timestamp,title,release_date,IMDb_URL,genres,age,gender,occupation,zip_code
0,196,242,3,881250949,Kolya (1996),24-Jan-1997,http://us.imdb.com/M/title-exact?Kolya%20(1996),Comedy,49,M,writer,55105
1,186,302,3,891717742,L.A. Confidential (1997),01-Jan-1997,http://us.imdb.com/M/title-exact?L%2EA%2E+Conf...,Crime|Film-Noir|Mystery|Thriller,39,F,executive,00000
2,22,377,1,878887116,Heavyweights (1994),01-Jan-1994,http://us.imdb.com/M/title-exact?Heavyweights%...,Children|Comedy,25,M,writer,40206
3,244,51,2,880606923,Legends of the Fall (1994),01-Jan-1994,http://us.imdb.com/M/title-exact?Legends%20of%...,Drama|Romance|War|Western,28,M,technician,80525
4,166,346,1,886397596,Jackie Brown (1997),01-Jan-1997,http://us.imdb.com/M/title-exact?imdb-title-11...,Crime|Drama,47,M,educator,55113


In [7]:
df = df.loc[df["genres"] != "unknown", :]

In [8]:
df.head()

,user_id,movie_id,rating,timestamp,title,release_date,IMDb_URL,genres,age,gender,occupation,zip_code
0,196,242,3,881250949,Kolya (1996),24-Jan-1997,http://us.imdb.com/M/title-exact?Kolya%20(1996),Comedy,49,M,writer,55105
1,186,302,3,891717742,L.A. Confidential (1997),01-Jan-1997,http://us.imdb.com/M/title-exact?L%2EA%2E+Conf...,Crime|Film-Noir|Mystery|Thriller,39,F,executive,00000
2,22,377,1,878887116,Heavyweights (1994),01-Jan-1994,http://us.imdb.com/M/title-exact?Heavyweights%...,Children|Comedy,25,M,writer,40206
3,244,51,2,880606923,Legends of the Fall (1994),01-Jan-1994,http://us.imdb.com/M/title-exact?Legends%20of%...,Drama|Romance|War|Western,28,M,technician,80525
4,166,346,1,886397596,Jackie Brown (1997),01-Jan-1997,http://us.imdb.com/M/title-exact?imdb-title-11...,Crime|Drama,47,M,educator,55113


É possível, no entanto, que o arquivo esteja em um formato que nenhuma biblioteca suporta. Nesse caso, podemos usar os métodos `readline` e `readlines` do Python...

In [16]:
with open(PATH) as file:

    # Ignora a primeira linha, pois ela é o cabeçalho.
    file.readline()

    # Armazena as linhas seguintes como lista de strings.
    lines = file.readlines()

...mas isso apenas separa o arquivo em linhas. 

In [17]:
# Para não sobrecarregar este notebook, vamos usar um
# slice para imprimir apenas as cinco primeiras linhas.
for line in lines[:5]:
    print(line)

0,196,242,3,881250949,Kolya (1996),24-Jan-1997,http://us.imdb.com/M/title-exact?Kolya%20(1996),Comedy,49,M,writer,55105

1,186,302,3,891717742,L.A. Confidential (1997),01-Jan-1997,http://us.imdb.com/M/title-exact?L%2EA%2E+Confidential+(1997),Crime|Film-Noir|Mystery|Thriller,39,F,executive,00000

2,22,377,1,878887116,Heavyweights (1994),01-Jan-1994,http://us.imdb.com/M/title-exact?Heavyweights%20(1994),Children|Comedy,25,M,writer,40206

3,244,51,2,880606923,Legends of the Fall (1994),01-Jan-1994,http://us.imdb.com/M/title-exact?Legends%20of%20the%20Fall%20(1994),Drama|Romance|War|Western,28,M,technician,80525

4,166,346,1,886397596,Jackie Brown (1997),01-Jan-1997,http://us.imdb.com/M/title-exact?imdb-title-119396,Crime|Drama,47,M,educator,55113



Para extrair dados dessas linhas, devemos entender o formato delas e implementar um *parsing* do zero.

In [18]:
# Para não sobrecarregar este notebook, vamos usar um
# slice para imprimir apenas as cinco primeiras linhas.
for line in lines[:5]:

    # Separa a linha em partes, usando
    # o caractere ',' como separador.
    parts = line.split(',')

    # Ignora o primeiro e último caracteres
    # de cada parte, para eliminar as aspas.
    parts = [part[1:-1] for part in parts]

    # Imprime a data, o mandante e o visitante.
    print(parts[2], parts[4], parts[5])

4 8125094 olya (1996
0 9171774 .A. Confidential (1997
7 7888711 eavyweights (1994
 8060692 egends of the Fall (1994
4 8639759 ackie Brown (1997


## Leitura dos dados em disco

Em muitas situações, por outro lado, o arquivo é grande demais para caber em memória. Nesses casos, é necessário processar esse arquivo linha por linha, sem nunca carregá-lo inteiro.

Dependendo do formato, o *parsing* dessas linhas pode ser feito por uma biblioteca. Novamente, esse é justamente o caso do CSV: podemos usar a função `reader` do módulo [csv](https://docs.python.org/3/library/csv.html).

In [19]:
with open(PATH) as file:
    reader = csv.reader(file)

    # Para não sobrecarregar este notebook, vamos usar um contador
    # e um break para imprimir apenas as cinco primeiras linhas.
    i = 0

    # Ignora a primeira linha, pois ela é o cabeçalho.
    next(reader)

    # Lê o arquivo linha por linha, sem carregá-lo inteiro na memória.
    for line in reader:

        # Imprime a data, o mandante e o visitante.
        print(line[2], line[4], line[5])

        # Para não sobrecarregar este notebook, vamos usar um contador
        # e um break para imprimir apenas as cinco primeiras linhas.
        i += 1
        if i == 5:
            break

242 881250949 Kolya (1996)
302 891717742 L.A. Confidential (1997)
377 878887116 Heavyweights (1994)
51 880606923 Legends of the Fall (1994)
346 886397596 Jackie Brown (1997)


Na pior das situações, o arquivo é grande e está em um formato não suportado. Nessa situação, infelizmente, devemos implementar praticamente tudo do zero.

In [20]:
with open(PATH) as file:

    # Para não sobrecarregar este notebook, vamos usar um contador
    # e um break para imprimir apenas as cinco primeiras linhas.
    i = 0

    # Ignora a primeira linha, pois ela é o cabeçalho.
    next(file)

    # Lê o arquivo linha por linha, sem carregá-lo inteiro na memória.
    for line in file:

        # Transforma a linha em uma lista de partes,
        # considerando o caractere ',' como separador.
        parts = line.split(',')

        # Ignora o primeiro e último caracteres
        # de cada parte, para eliminar as aspas.
        parts = [part[1:-1] for part in parts]

        # Imprime a data, o mandante e o visitante.
        print(parts[2], parts[4], parts[5])

        # Para não sobrecarregar este notebook, vamos usar um contador
        # e um break para imprimir apenas as cinco primeiras linhas.
        i += 1
        if i == 5:
            break

4 8125094 olya (1996
0 9171774 .A. Confidential (1997
7 7888711 eavyweights (1994
 8060692 egends of the Fall (1994
4 8639759 ackie Brown (1997


## Construção de um novo grafo

Para construir e analisar grafos, vamos usar a biblioteca [graph-tool](https://graph-tool.skewed.de/), com alguns adicionais disponíveis no módulo `graph_tool_extras`.

O uso básico dessa plataforma é simples:

* para criar um grafo, basta chamar a classe `gte.Graph`; *(o parâmetro booleano `directed`, que é falso por padrão, define se ele é dirigido)*
 
* para adicionar um vértice a esse grafo, basta chamar o método `add_vertex_by_id`, passando o identificador desse vértice; *(inteiro ou string)*

* para adicionar uma aresta, basta chamar o método `add_edge_by_ids`, passando os identificadores da origem e do destino.

Ou seja, para construir o primeiro exemplo do primeiro notebook, basta rodar o código abaixo.

In [7]:
g = gte.Graph(directed=False) # igual a gte.Graph(), pois directed é falso por padrão

genres = [
    "Action", "Adventure", "Animation", "Children", "Comedy", "Crime",
    "Documentary", "Drama", "Fantasy", "Film-Noir", "Horror", "Musical",
    "Mystery", "Romance", "Sci-Fi", "Thriller", "War", "Western"
]

for genre in genres:
    g.add_vertex_by_id(genre)

for movie_id in set(df["movie_id"]):
    g.add_vertex_by_id(movie_id)

    genres_by_movie = df.loc[df["movie_id"] == movie_id, :].values.tolist()[0][7].split("|")

    for genre in genres_by_movie:
        g.add_edge_by_ids(movie_id, genre)


## Iteração e acesso a vértices

Para iterar sobre os vértices de um grafo, basta chamar o método `vertices`.

Para acessar o identificador de um vértice, basta chamar o método `id`.

In [12]:
for u in g.vertices():
    print(u.id())

a
b
c
d


Para acessar um vértice específico, basta chamar o método `vertex_by_id`, passando o identificador desse vértice.

In [13]:
u = g.vertex_by_id('a')

## Iteração e acesso a arestas

Para iterar sobre as arestas de um grafo, basta chamar o método `edges`.

Para acessar a origem de uma aresta, basta chamar o método `source`.

Para acessar o destino de uma aresta, basta chamar o método `target`.

In [14]:
for e in g.edges():
    print(e.source().id(), e.target().id())

a b
a c
b c
b d
c b
c d


Para acessar uma aresta específica, basta chamar o método `edge_by_ids`, passando os identificadores da origem e do destino.

In [15]:
e = g.edge_by_ids('a', 'b')

## Armazenamento de valores

A plataforma permite armazenar valores em cada vértice, em cada aresta e no próprio grafo. Esses valores podem ser qualquer coisa: inteiros, strings, listas, dicionários...

### Propriedades de vértice

Um vértice é mais ou menos como um dicionário: para armazenar um valor nele, devemos associar esse valor a uma chave.

A atribuição abaixo, por exemplo, associa o inteiro `1` à chave `'x'` do vértice `u`.

``` python
u['x'] = 1
```

Porém, para evitar problemas de consistência, não é permitido que um vértice tenha uma chave e outro não. No exemplo acima, há somente duas possibilidades: ou *nenhum* vértice tem a chave `'x'` (ou seja, a atribuição com certeza daria erro), ou *todos* têm (ou seja, a atribuição com certeza iria funcionar). Nesse caso, dizemos que o grafo tem uma **propriedade de vértice** *(vertex property)* chamada `'x'`.

Para adicionar essa propriedade de vértice ao grafo, devemos chamar o método `add_vp`, passando sua chave.

In [16]:
g.add_vp('x')

Se a propriedade existe, tanto a leitura quanto a escrita de valores são análogas às de um dicionário.

In [17]:
u['x'] = 1

u['x']

1

### Propriedades de aresta

Assim como um grafo pode ter uma propriedade de vértice, ele também pode ter uma **propriedade de aresta** *(edge property).* Para adicioná-la, basta chamar o método `add_ep`.

In [18]:
g.add_ep('y')

Novamente, se a propriedade existe, tanto a leitura quanto a escrita de valores são análogas às de um dicionário.

A atribuição abaixo associa a string `'s'` à chave `'y'` da aresta `e`.

In [19]:
e['y'] = 's'

e['y']

's'

### Propriedades de grafo

Por fim, um grafo pode ter uma propriedade dele próprio, ou seja, uma **propriedade de grafo** *(graph property).* Para adicionar, basta chamar `add_gp`.

In [20]:
g.add_gp('z')

A atribuição abaixo associa a lista `[1, 's']` à chave `'z'` do grafo `g`.

In [21]:
g['z'] = [1, 's']

g['z']

[1, 's']

## Salvamento de grafo em arquivo

Para salvar um grafo em um arquivo, basta chamar a função `gte.save`, passando o grafo e o caminho do arquivo.

Se o grafo tem propriedades de vértice, aresta ou dele próprio, elas também são salvas.

In [22]:
gte.save(g, 'grafo.net.gz')

## Conclusão

Vamos usar as seções anteriores para construir a seguinte rede:

* os vértices são os times que participaram do campeonato de um ano específico; *(os identificadores são os nomes desses times)*
* uma aresta de A a B indica que o time A ganhou do time B em pelo menos uma partida desse campeonato.

Como a relação "A ganhou de B" não é simétrica, devemos criar uma rede dirigida.

In [23]:
g = gte.Graph(directed=True)

Usando propriedades, vamos armazenar três informações adicionais na rede:

* saldo de gols de cada time; *(propriedade de vértice)*
* total de vitórias de cada um dos times sobre cada um dos outros; *(propriedade de aresta)*
* ano escolhido. *(propriedade de grafo)*

In [24]:
g.add_vp('saldo')
g.add_ep('total')
g.add_gp('ano')

Para usar dados específicos, vamos escolher o ano de 2022.

In [25]:
YEAR = 2022

g['ano'] = YEAR

Antes de ler o arquivo, vamos definir uma função auxiliar que recebe um grafo e um identificador e:

* se um vértice com esse identificador já existe no grafo, simplesmente devolve esse vértice;

* senão, adiciona um vértice com esse identificador ao grafo, inicializa o saldo desse vértice e o devolve.

Para isso, vamos usar dois fatos úteis:

1. se um vértice com o identificador não existe, o método `vertex_by_id` devolve `None`;

2. o método `add_vertex_by_id` devolve o vértice adicionado.

In [26]:
def get_or_add_vertex(g, id):
    u = g.vertex_by_id(id)
    if u is None:
        u = g.add_vertex_by_id(id)
        u['saldo'] = 0
    return u

Vamos definir também uma função auxiliar que recebe um grafo e dois identificadores e:

* se os vértices com esses identificadores já estão conectados no grafo, simplesmente devolve a aresta correspondente a eles;

* senão, adiciona uma aresta correspondente a esses identificadores ao grafo, inicializa o total dessa aresta e a devolve.

Vamos usar dois fatos análogos aos da função anterior:

1. se uma aresta correspondente aos identificadores não existe, o método `edge_by_ids` devolve `None`;

2. o método `add_edge_by_ids` devolve a aresta adicionada.

In [27]:
def get_or_add_edge(g, winner, loser):
    e = g.edge_by_ids(winner, loser)
    if e is None:
        e = g.add_edge_by_ids(winner, loser)
        e['total'] = 0
    return e

Agora estamos prontos para ler o arquivo e construir a rede.

In [28]:
with open(PATH) as file:
    reader = csv.reader(file)

    # Ignora a primeira linha, pois ela é o cabeçalho.
    next(reader)

    # Lê o arquivo linha por linha, sem carregá-lo inteiro na memória.
    for line in reader:

        # Considera apenas partidas de 2022.
        date = line[2]
        index = date.rfind('/')
        year = int(date[(index + 1):])
        if year == YEAR:

            # Lê os nomes dos times mandante e visitante.
            home = line[4]
            guest = line[5]

            # Lê os placares dos times mandante e visitante.
            score_home = int(line[12])
            score_guest = int(line[13])

            # Obtém vértices, adicionando se não existirem.
            u = get_or_add_vertex(g, home)
            v = get_or_add_vertex(g, guest)

            # Atualiza saldos dos vértices.
            u['saldo'] += score_home
            v['saldo'] += score_guest

            # Considera apenas partidas em que não houve empate.
            if score_home != score_guest:

                # Obtém aresta, adicionando se não existir.
                if score_home < score_guest:
                    e = get_or_add_edge(g, guest, home)
                else:
                    e = get_or_add_edge(g, home, guest)

                # Atualiza total da aresta.
                e['total'] += 1

Para concluir, basta salvar a rede em um arquivo.

In [8]:
gte.save(g, 'movies.net.gz')

<div class="alert alert-block alert-info">
   <strong>DICA:</strong> Você pode clicar na aba
   
   à esquerda para navegar por seção.
</div>

In [10]:
import netpixi

from graph_tool import draw

In [12]:
g = gte.load("movies.net.gz")

layout = draw.sfdp_layout(g)

gte.move(g, layout)

gte.save(g, "movies_reposicionado.net.gz")

In [ ]:
netpixi.render("movies_reposicionado.net.gz")

In [15]:
len(genres) + len(set(df["movie_id"]))

1698

In [ ]:
len

In [16]:
g.num_edges()

2891

In [18]:
g.num_vertices()

1698